In [32]:
##########################Data preparation #####################################
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import torch.utils.data
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

number_of_epochs = 10

def lines_to_np_array(lines):
    return np.array([[i for i in line.split()] for line in lines])
with open('PATCH.amat') as f:
    lines = f.readlines()
x_train = lines_to_np_array(lines).astype('float32')
idx = np.random.permutation(len(x_train))

y_train = x_train[idx, 784:]
x_train = x_train[idx, :784]

X_train, X_test, Y_train, Y_test = train_test_split(x_train, y_train, test_size=0.2)
#X_valid, X_test, Y_valid, Y_test = train_test_split(X_test, Y_test, test_size=0.5)

X_train_final = []
X_test_final = []
#X_valid_final = []

train_accuracy = []
#valid_accuracy = []

for i in range(0, len(X_train)):
    X_train_final.append(torch.Tensor(X_train[i].reshape((1, 28, 28))))
for i in range(0, len(X_test)):
    X_test_final.append(torch.Tensor(X_test[i].reshape((1, 28, 28))))
#for i in range(0, len(X_valid)):
    #X_valid_final.append(torch.Tensor(X_valid[i].reshape((1, 28, 28))))
    
print("Train data", (X_train_final[0].shape), Y_train.shape)
print("Test data", (X_test_final[0].shape), Y_test.shape)
#print("Validation data", (X_valid_final[0].shape), Y_valid.shape)

####################################Pytorch loader##############################
batch_size = 500
class custom_dataset(torch.utils.data.dataset.Dataset):
    def __init__(self, data, labels, transforms=None):
        self.data = data
        self.labels = labels
        self.transforms = transforms

    def __getitem__(self, index):   
        dat = self.data[index]
        if self.transforms is not None:
            dat = self.transforms(dat)
        return (dat,self.labels[index])
   
    def __len__(self):
        return len(self.data)
      
transform_train = transforms.Compose([transforms.ToPILImage(), transforms.ToTensor()])
transform_test = transforms.Compose([transforms.ToPILImage(), transforms.ToTensor()])

train = custom_dataset(X_train_final,torch.LongTensor(Y_train.reshape(len(Y_train))), transforms=transform_train)
train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True, num_workers=0)

#validation = custom_dataset(X_valid_final,torch.LongTensor(Y_valid.reshape(len(Y_valid))), transforms=transform_test)
#validation_loader = torch.utils.data.DataLoader(validation, batch_size=batch_size, shuffle=False, num_workers=0)

test = custom_dataset(X_test_final,torch.LongTensor(Y_test.reshape(len(Y_test))), transforms=transform_test)
test_loader = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=False, num_workers=0)
######################### DESIGN CNN MODEL #####################################
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
  
        self.conv1 = nn.Conv2d(1, 32, 3, padding=0) 
        self.conv2 = nn.Conv2d(32, 64, 3, padding=0)        
        self.pool1 = nn.MaxPool2d(2, 2)
        self.drop1 = nn.Dropout2d(p=0.3)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 12 * 12, 1024)
        self.drop2 = nn.Dropout2d(p=0.5)
        self.fc2 = nn.Linear(1024, 512)
        self.drop3 = nn.Dropout2d(p=0.5)
        self.fc3 = nn.Linear(512, 2)
        self.bn1 = nn.BatchNorm2d(32)
        self.bn2 = nn.BatchNorm2d(64)
        self.bn4 = nn.BatchNorm1d(1024)
        self.bn5 = nn.BatchNorm1d(512)
        
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.bn1(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.bn2(x)
        x = self.pool1(x)
        x = x.view(-1, 64 * 12 * 12)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.bn4(x)
        x = self.drop2(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.bn5(x)
        x = self.drop3(x)
        x = self.fc3(x)
        return x

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = ConvNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adagrad(model.parameters(), lr=0.01, weight_decay=0.015)
################################ Train, Validation and test the model###########
for epoch in range(number_of_epochs):
    correct = 0.
    total = 0.
    model.train()
    running_loss_train = 0.0
    running_loss_test = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs.to(device))
        loss = criterion(outputs, labels.to(device))
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels.to(device)).sum().item()
    train_accuracy.append(correct / total)
    
    #correct = 0.
    #total = 0.
    #model.eval()
    #with torch.no_grad():
        #for data in validation_loader:
            #images, labels = data
            #outputs = model(images.to(device))
            #_, predicted = torch.max(outputs.data, 1)
            #total += labels.size(0)
            #correct += (predicted == labels.to(device)).sum().item()
            #loss = criterion(outputs, labels.to(device))
    #valid_accuracy.append(correct/total)
    
    print('Epoch:', epoch)
    print('Train accuracy:', train_accuracy[epoch]*100)
    #print('Validation accuracy:', valid_accuracy[epoch]*100)
torch.save(model.state_dict(), r"finalmodel.pt")
###################Plot learning curves ########################################
#plt.figure()
#plt.plot(range(number_of_epochs),train_accuracy, 'sk-', label='Train')
#plt.plot(range(number_of_epochs),valid_accuracy, 'sr-', label='Valid')
#plt.xlabel('Epoch')
#plt.ylabel('Accuracy')
#plt.legend()
#torch.cuda.empty_cache()
#plt.show()
################################################################################
#######################Test the model with Unseen data##########################
correct = 0.
total = 0.
model.eval()
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = model(images.to(device))
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels.to(device)).sum().item()
        loss = criterion(outputs, labels.to(device))
        
print("Final test accuracy", (correct/total)*100)
################################################################################

Train data torch.Size([1, 28, 28]) (40000, 1)
Test data torch.Size([1, 28, 28]) (10000, 1)
Epoch: 0
Train accuracy: 76.19749999999999
Epoch: 1
Train accuracy: 84.89
Epoch: 2
Train accuracy: 90.3425
Epoch: 3
Train accuracy: 93.4375
Epoch: 4
Train accuracy: 95.3225
Epoch: 5
Train accuracy: 96.0375
Epoch: 6
Train accuracy: 96.69250000000001
Epoch: 7
Train accuracy: 96.72500000000001
Epoch: 8
Train accuracy: 97.36
Epoch: 9
Train accuracy: 97.45
Final test accuracy 97.47


In [2]:
print("x Shape", x.shape)# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

tcmalloc: large alloc 1073750016 bytes == 0x589ae000 @  0x7f93dde652a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641


In [0]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

In [0]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

In [0]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch